# Project_1_ITSolera_DL.

AI-Enabled Smart Agriculture System.

Project Lead: DL Team Zeta at ITSOLERA PVT LTD.

In [24]:
#pip install torch

##### Step 1: Importing the required libraries and modules.

In [29]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
print(torch.__version__)
print(torch.cuda.is_available())

2.4.0
False


##### Step 2: Defining Transformations to be applied to the images, include resizing, converting to tensor, and normalizing.



In [26]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Required by ResNet18
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])

##### Step 3: Data Loading.

In [28]:
dataset = ImageFolder(root='D:\\2024 DS and ML Course_Ammar\\IT Solera DL\\Project 1\\Images datasets\\Zeta images dataset2_1gb', transform=transform)

##### Step 4: Spliting the Dataset for Training and Testing.

In [30]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset  = torch.utils.data.random_split(dataset, [train_size, val_size])

In [34]:
print(f'train_size: {train_size}')
print(f'val_size: {val_size}')
print(f'train_dataset: {train_dataset}')
print(f'val_dataset: {val_dataset}')

train_size: 51421
val_size: 12856
train_dataset: <torch.utils.data.dataset.Subset object at 0x000002C64EC69850>
val_dataset: <torch.utils.data.dataset.Subset object at 0x000002C64ECF6990>


##### Step 5: Creating data loaders for the Training and Testing sets to handle batching and shuffling.

In [39]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
print(f'train_loader: {train_loader}')
print(f'val_loader: {val_loader}')

train_loader: <torch.utils.data.dataloader.DataLoader object at 0x000002C64ECD8C10>
val_loader: <torch.utils.data.dataloader.DataLoader object at 0x000002C64CE101D0>


##### Step 6: Defining the Model.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
	def __init__(self):
		super(SimpleCNN, self).__init__()
		self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
		self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
		self.fc1 = nn.Linear(64 * 56 * 56, 128)
		self.fc2 = nn.Linear(128, 10)  # Assuming 10 classes

	def forward(self, x):
		x = F.relu(self.conv1(x))
		x = F.max_pool2d(x, 2)
		x = F.relu(self.conv2(x))
		x = F.max_pool2d(x, 2)
		x = x.view(x.size(0), -1)
		x = F.relu(self.fc1(x))
		x = self.fc2(x)
		return x

model = SimpleCNN()

##### Step 7: Defining Loss Fun and Optimizer.

In [ ]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

##### Step 8: Training the Model.

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
	model.train()
	running_loss = 0.0
	for inputs, labels in train_loader:
		optimizer.zero_grad()
		outputs = model(inputs)
		loss = criterion(outputs, labels)
		loss.backward()
		optimizer.step()
		running_loss += loss.item()
	
	print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

##### Step 9: Validate the Model.

In [ ]:
model.eval()
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
	for inputs, labels in val_loader:
		outputs = model(inputs)
		loss = criterion(outputs, labels)
		val_loss += loss.item()
		_, predicted = torch.max(outputs.data, 1)
		total += labels.size(0)
		correct += (predicted == labels).sum().item()

print(f'Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%')

##### Step 10: Saving the model.

In [ ]:
torch.save(model.state_dict(), 'model.pth')

##### Step 11: Evaluating (Testing) the Modle.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Loss: {test_loss/len(test_loader)}, Accuracy: {100 * correct / total}%')